In [1]:
import os
import sys

def path_is_in_pythonpath(path):
    path = os.path.normcase(path)
    return any(os.path.normcase(sp) == path for sp in sys.path)

path_project = os.path.dirname(os.path.abspath(os.getcwd()))
if not path_is_in_pythonpath(path_project):
    sys.path.append(path_project)

In [2]:
import json
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import matplotlib.colors as mcolors

In [3]:
# read json network
with open('../data/prototypes/adjacency_list_03.csv.cyjs', 'r') as f:
    elements = json.loads(f.read())

In [4]:
# create linear gradient based on color values
def generate_correlation_color(value_corr, value_min=-1, value_max=1,
                               color_min = '#1E90FF', color_mid = 'white', color_max = '#FF4500'):

    colormap = mcolors.LinearSegmentedColormap.from_list(
        'correlation_cmap', [color_min, color_mid, color_max])
    normalized_value = (value_corr - value_min) / (value_max - value_min)
    color_hex = mcolors.to_hex(colormap(normalized_value))
    return color_hex


In [10]:
# create new json file
df_vglut = pd.read_csv('../data/prototypes/vGlut1_geneInfo.csv', sep = ',', index_col=1)
df_vgat = pd.read_csv('../data/prototypes/vGAT_geneInfo.csv', sep = ',', index_col=1)
df_modules = pd.read_csv('../data/prototypes/syndive_module_colors.csv', sep=',', index_col=0)

df = pd.DataFrame({'gene': df_vglut['GeneName'],
                'protein': df_vglut.index,
                'module': df_vglut['Module'],
                'vGlut': df_vglut['GS.vGAT'],
                'vGat': df_vgat['GS.vGAT']})
df.set_index('protein', inplace=True)
df.loc['TdTomato', 'gene'] = 'TdTomato'

x_filter = 1300
y_filter = -1400

node_filter = []
node_map = {}
el = elements['elements']
cytograph = {'nodes':[], 'edges':[]}
for node in el['nodes']:

    x = node['position']['x']
    y = node['position']['y']

    if (x > x_filter) and (y < y_filter):
        node_filter.append(node['data']['id'])
        print(x,y)
        continue

    protein = node['data']['Proteins']
    gene = df.loc[protein, 'gene']
    module = df.loc[protein, 'module']
    node_map[node['data']['id']] = gene
    cytonode = {
        'data':{
            'id': node['data']['id'],
            #'SUID': node['data']['SUID'],
            'protein': protein,
            'gene': gene,
            'module': module,
            'color_module': df_modules.loc[module,'color'],
            'vglut': node['data']['GS_vGlut1'],
            'color_vglut': generate_correlation_color(node['data']['GS_vGlut1']),
            'vgat': node['data']['GS_vGAT'],
            'color_vgat': generate_correlation_color(node['data']['GS_vGAT']),
        },
        'position':{'x':x, 
                    'y':y},
        'selected': False,
        'selectable': True,
        'locked': False,
        'grabbable': True}
    cytograph['nodes'].append(cytonode)

for edge in el['edges']:
    
    if (edge['data']['source'] in node_filter) or (edge['data']['target'] in node_filter):
       continue

    cytoedge = {
        'data': {
            'id': edge['data']['id'],
            #'SUID': edge['data']['SUID'],
            'source': edge['data']['source'],
            'target': edge['data']['target'],
            'source_gene': node_map[edge['data']['source']],
            'target_gene': node_map[edge['data']['target']],
            'freq': edge['data']['Freq']
        },
        'selected': False,
        'selectable': False
    }
    cytograph['edges'].append(cytoedge)

# write to file
json_object = json.dumps(cytograph, indent=4)
with open('../data/prototypes/cytoscape_info.json', "w") as json_file:
    json_file.write(json_object)

1518.8625350974241 -1401.4538344307755
1521.1841255210081 -1509.1931837960099
1621.1841255210081 -1401.4538344307755
1623.505715944592 -1509.1931837960099
1723.505715944592 -1401.4538344307755
1725.827306368176 -1509.1931837960099
1825.827306368176 -1401.4538344307755
1828.14889679176 -1509.1931837960099
1928.14889679176 -1401.4538344307755
1930.470487215344 -1509.1931837960099
2030.470487215344 -1401.4538344307755
2032.792077638928 -1509.1931837960099
2132.7920776389283 -1401.4538344307755
2135.1136680625123 -1509.1931837960099
2235.1136680625123 -1401.4538344307755
2237.4352584860962 -1509.1931837960099
2337.4352584860962 -1401.4538344307755
2339.7568489096802 -1509.1931837960099
2439.7568489096802 -1401.4538344307755
2442.078439333264 -1509.1931837960099
2542.078439333264 -1401.4538344307755
2544.400029756848 -1509.1931837960099
2644.400029756848 -1401.4538344307755
2646.721620180432 -1509.1931837960099
1311.6177135489622 -1509.1931837960099
1418.8625350974241 -1498.5539015694474
13

In [22]:
# create nodes and edges for plotly
node_id = []
node_x = []
node_y = []
for node in elements['elements']['nodes']:
    node_id.append(int(node['data']['id']))
    node_x.append(-node['position']['x'])
    node_y.append(-node['position']['y'])

edge_x = []
edge_y = []
for edge in elements['elements']['edges']:
    source = int(edge['data']['source'])
    target = int(edge['data']['target'])
    edge_x.append(node_x[node_id.index(source)])
    edge_x.append(node_x[node_id.index(target)])
    edge_x.append(None)
    edge_y.append(node_y[node_id.index(source)])
    edge_y.append(node_y[node_id.index(target)])
    edge_y.append(None)


In [21]:
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='#880'),
    hoverinfo='none',
    mode='lines')

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        line_width=2))

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

NameError: name 'edge_x' is not defined

In [23]:
fig = px.scatter(x=node_x, y=node_y)
fig.add_scatter(x=edge_x, y=edge_y)
fig.update_xaxes(
    constrain="domain",  # meanwhile compresses the xaxis by decreasing its "domain"
)
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1
)
fig.show()

In [56]:
print(edge_x)

[-361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.86251106041277, None, -361.86251106041277, -361.8625110604127